In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for deap: filename=deap-1.4.0-cp38-cp38-macosx_10_9_x86_64.whl size=104826 sha256=d2db7b830e5068df68146b05db8fddfab968ce3a16cd47bb0bbfc218a84ab4dd
  Stored in directory: /Users/diongarman/Library/Caches/pip/wheels/54/63/b5/4e1a5ec0e78d9b9c3f87c71821fd47d25f7b6f64645f53d9dd
Successfully built deap


In [3]:

def typefactor(data, team):
    tp = [data['planets'].loc[ti1]['type'] for ti1 in team]
#     print('tp: ', tp)
    nt = [tp.count(i) for i in tp]
#     print('nt: ', nt)
    return [.95**(i-1) for i in nt]

def getparams(data, team):
    tf  = typefactor(data, team)
    p = sum(data['planets'].loc[i][['p0','p1','p2']].values*f for i, f in zip(team, tf) )
#     print('p:', p)
    return p


#obj1
def resoltime(data, team):
    e = getparams(data, team)
#     print('e:',  e) 
#     print('e:',  e==0)#e.g.[False False False]
    t = 99999 if (e==0).any() else sum(data['difficulty']/e)
    return t
#obj1 or is this a constraint
def resources(data, team):
    return sum(data['planets'].loc[i]['resources'] for i in team)

#random team - should beat this
def rndteam(data, n=6, fix=''):
    rs=9999
    while not rs<=data['maxresources']:
        team  = [np.random.choice(data['leaderlst'])] if  fix=='' else [fix] 
        team += list(np.random.choice(data['workerlst'], n-1, replace=False))
        rs    = resources(data, team)
    return sorted(team)
#also develop a heuristic
#try beat a heuristic

In [4]:
df     = pd.read_excel('audiverse_dataset.xlsx', skiprows=0, sheet_name='dataset')
skft   = pd.read_excel('audiverse_dataset.xlsx', skiprows=1, index_col='type', sheet_name='skills_factor')
# df.head()
# skft.head()

#challenges correspond to 'indexing, 'quantum computer' and 'ai'
for challenge in range(3):
    df['p'+str(challenge)] = df[['type', 'resources']].apply(lambda row: skft.loc[row.type][challenge]*row.resources, axis=1) #rowwise

data   = {'planets'      :df, 
          'skills'       :skft, 
          'maxresources' : 900,
          'difficulty'   :[400, 800, 1600],
          'leaderlst'    :list(df[df['leadership']=='yes']['idx']),
          'workerlst'    :list(df[df['leadership']=='no']['idx']),
          'typelst': list(skft['indexing'].index)
         }

df.head()

,idx,planet,leadership,type,resources,p0,p1,p2
0,0,Xecconus,yes,developer,167.61,67.044,67.044,0.000
1,1,Debaitera,yes,auditor,233.81,93.524,70.143,23.381
2,2,Xingone,yes,quant,217.86,0.000,87.144,87.144
3,3,Odrov,yes,developer,544.97,217.988,217.988,0.000
4,4,Dualea,yes,validator,220.00,88.000,22.000,66.000


In [5]:
auditors = list(df[(df['leadership']=='no') & (df['type'] == 'auditor')]['idx'])
engineers = list(df[(df['leadership']=='no') & (df['type'] == 'engineer')]['idx'])
validators = list(df[(df['leadership']=='no') & (df['type'] == 'validator')]['idx'])
analysts = list(df[(df['leadership']=='no') & (df['type'] == 'analyst')]['idx'])
developers = list(df[(df['leadership']=='no') & (df['type'] == 'developer')]['idx'])
pmos = list(df[(df['leadership']=='no') & (df['type'] == 'pmo')]['idx'])
data_miners = list(df[(df['leadership']=='no') & (df['type'] == 'data miner')]['idx'])


ITMS = [data['leaderlst'],  data_miners, engineers,  analysts, developers, pmos ]

In [6]:
from collections import Counter

#helpers

def normalize(value, lower_bound, upper_bound):
    return (value - lower_bound) / (upper_bound - lower_bound)

def custom_eaSimple(population, toolbox, cxpb, ngen, stats=None, halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals', 'stagnation', 'mutation', 'tm'] + (stats.fields if stats else [])

    # Evaluate the individuals in initial population
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), stagnation=0, mutation=0, **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    stagnation = 0
    stagnation_threshold = 3
    mutpb = 0.25  # Initialize mutation probability
    stagnation_niching_flag = stagnation > 50
    niching_n_times = 25


    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        
        #put offspring/population into brackets with different strategies

        # Evaluate the individuals
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            
            
            
            ind.fitness.values = fit
                    
            if stagnation_niching_flag:
            #perform niching
                ind.fitness.values = shared_fitness(ind, offspring, 4)
                stagnation_niching_flag = niching_n_times > 0
#                 print('niching performed')
                
            else:
                niching_n_times = 25 #reset
                stagnation_niching_flag = stagnation > 50
        
        niching_n_times -= 1        


        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)
            type_makeup = [df.iloc[i]['type'] for i in halloffame[0]] #makeup of the best
#             print(type_makeup)
            
#             task_scores = getparams(data, halloffame[0])
#             ratios1 = np.array([task_scores[i] / task_scores[i+1] for i in range(len(task_scores) - 1)])
            
        
            

        # Replace the current population by the offspring
        population[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), mutation=mutpb,stagnation = stagnation, tm=len(set(type_makeup)), **record)
        if verbose:
            print(logbook.stream)

        # Check if fitness is improving
        curr = logbook.select('min')[gen]
        prev = logbook.select('min')[gen-1]
        
        
        
        
#         print('gen: ', gen)
#         print('{curr} >= {prev}'.format(curr=curr, prev=prev))
        stagnated = curr >= prev
#         print('stagnated?', stagnated)
        if gen > 1 and stagnated:
            

            lower_bound = 0
            upper_bound = max(logbook.select("IQR"))
            curriqr = logbook.select('IQR')[gen]
            
            mutpb = 1 - normalize(curriqr, lower_bound, upper_bound)
            

        
            stagnation += 1
        else:
            stagnation = 0
            
        

        # Increase mutation rate if fitness is not improving
        rate_of_change = 0.01
        if stagnation > stagnation_threshold:
            mutpb = max(.5, mutpb )  # increase mutation rate as function of stagnation
            #cxpb = min(1.0, cxpb + rate_of_change)  # increase crossover rate as function of stagnation

        else:
            mutpb = min(0.02, mutpb)  # decrease mutation rate slightly
            #cxpb = max(0, cxpb - rate_of_change)  # increase crossover rate as function of stagnation
            

    return population, logbook


In [7]:
# TEAM_SIZE = 6

POP_SIZE = 128

P_CROSSOVER = 0.5

# P_MUTATION = 0.1

MAX_GENERATIONS = 10000

PENALTY = 0 #may have to figure a smarter penalty

CAPACITY = 900

# RANDOM_SEED = 42

In [8]:
kuba_best_ratio = [0.59706455, 0.70709521]
my_best_invalid_ratio = [0.58923431, 0.7242129]
my_best_ratio = [0.59553237, 0.71009381]
ideal = [0.5, 0.5]

ratio_wiggle_rooms = [(((abs(0.6-ideal[0])/(ideal[0]))) ), (((abs(0.7-ideal[1])/(ideal[1]))) )]
ideal = np.array([0.5, 0.5])

def e_distance(list1, list2):
    """Distance between two vectors."""
    squares = [(p-q) ** 2 for p, q in zip(list1, list2)]
    return sum(squares) ** .5

# Define the fitness function
def fitness(individual):

    restime   = resoltime(data, individual)
    
    return restime,

def feasible(individual):
    
    leaders_list = [i for i in individual if 0 <= i <= 63]
    weight = resources(data, individual)
    task_scores = getparams(data, individual)
    ratios1 = np.array([task_scores[i] / task_scores[i+1] for i in range(len(task_scores) - 1)])
    
    good_ratio = np.all(np.isclose(ratios1, ideal, rtol=ratio_wiggle_rooms))
    
    if weight > CAPACITY:
        return False
    if len(leaders_list) > 1:
        return False
    if not good_ratio:
        return False
    return True

def distance(individual):
    weight = resources(data, individual)
    restime   = resoltime(data, individual)
    task_scores = getparams(data, individual)
    ratios1 = np.array([task_scores[i] / task_scores[i+1] for i in range(len(task_scores) - 1)])
    punish_factor = e_distance(ratios1, ideal) * .1
#     punish_factor = abs(sum(ratios1) - 1)
#     print('punish factor: ', punish_factor)

    return (0.045*abs(CAPACITY-weight)) + restime + punish_factor



In [9]:
#niching

def shared_fitness(individual, population, dist_threshold):
    
    count = sum(1 for other in population if hamming_distance(individual, other) < dist_threshold)
    return (individual.fitness.values[0] + max(0, count)*individual.fitness.values[0]*0.02,) #add an extra 2% for each neighbour

def hamming_distance(string1, string2):
    string1 = sorted(string1)
    string2 = sorted(string2)
    if len(string1) != len(string2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(el1 != el2 for el1, el2 in zip(string1, string2))



In [10]:

def custom_mutate(individual, indpb):
    # Determine the index of the single value in the range [0, 63]
    indices = [i for i in range(len(individual)) if 0 <= individual[i] <= 63]
    if indices:
        special_index = indices[0]
    else:
        special_index = random.randint(0, len(individual) - 1)
        individual[special_index] = random.randint(0, 63)
    
    for i in range(len(individual)):
        if random.random() < indpb:
            if i == special_index:
                individual[i] = random.randint(0, 63)  # Mutate within the range [0, 63]
            else:
                individual[i] = random.randint(64, 959)  # Mutate within the range [64, 959]
    
    return individual,

def type_constrained_mutate(individual,  indpb):
    # For each element in the individual
    for i in range(len(individual)):
        # If the random condition for mutation is met (e.g., 10% chance)
        if random.random() < indpb:
            # Mutate the element to a random choice from the corresponding list
            individual[i] = random.choice(ITMS[i])
    return individual,


    

In [11]:
def uniform_crossover(ind1, ind2, indpb):
    for i in range(len(ind1)):
        if random.random() < indpb:
            ind1[i], ind2[i] = ind2[i], ind1[i]
    return ind1, ind2

In [12]:
# def leader_attr():
#     return np.random.choice(data['leaderlst'])
# def worker_attr():
#     return np.random.choice(data['workerlst'])



# def create_individual():
#     team = creator.Individual([toolbox.attr_leader()] + [toolbox.attr_worker() for _ in range(5)])

#     return team


        
# def leader_attr():
#     scientist_leaders = list(df[(df['leadership']=='yes') & (df['type'] == 'scientist')]['idx'])
#     return np.random.choice(scientist_leaders)

def leader_attr():
    return 33

def worker_attr(t):
    worker = list(df[(df['leadership']=='no') & (df['type'] == t)]['idx'])
    return np.random.choice(worker)


def create_individual():
    type_makeup = ['data miner', 'engineer','analyst', 'developer','pmo']
    team = creator.Individual([toolbox.attr_leader()] + [toolbox.attr_worker(t) for t in type_makeup])

    return team

In [13]:
from deap import base, creator, tools, algorithms
import random


# Set up the genetic algorithm

#create types
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

#initialisation
toolbox = base.Toolbox()

toolbox.register("attr_leader", leader_attr)
toolbox.register("attr_worker", worker_attr)
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#Operators
toolbox.register("evaluate", fitness)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, PENALTY, distance))

#reproduction
toolbox.register("mate", uniform_crossover,indpb=.333)
toolbox.register("mutate", type_constrained_mutate, indpb=0.333)
#selection
toolbox.register("select", tools.selTournament, tournsize=5)

#Statistics and Data Collection
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("Q1", lambda x: np.percentile(x, 25))
stats.register("Q2", lambda x: np.percentile(x, 50))
stats.register("Q3", lambda x: np.percentile(x, 75))
stats.register("min", lambda x: np.min(x))
stats.register("max", lambda x: np.max(x))
stats.register("IQR", lambda x: np.percentile(x, 75)-np.percentile(x, 25))


#run evolution algorithm
pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(150)
pop, log = custom_eaSimple(pop, toolbox, cxpb=P_CROSSOVER, ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    
# Print the best individuals
for i in hof:

    teamrsrcs = resources(data, i)
    restime   = resoltime(data, i)
    type_makeup = [df.iloc[ti]['type'] for ti in i]
    print( 'The team {} has {:6.2f} resources,'.format(i, teamrsrcs) )
    print( 'and solves the three challenges in {} years.'.format(restime) )
    print( 'and has the type makeup: {}'.format(type_makeup) )

gen	nevals	stagnation	mutation	tm	avg    	Q1     	Q2     	Q3     	min    	max    	IQR    
0  	128   	0         	0       	  	12.7915	11.0871	12.1751	13.6618	9.97413	22.9052	2.57471
1  	86    	0         	0.25    	6 	12.1319	10.4853	10.926 	11.8513	9.92951	32.4957	1.36601
2  	77    	0         	0.02    	6 	10.8088	10.0508	10.35  	10.7878	9.89263	27.7149	0.736991
3  	61    	0         	0.02    	6 	10.3   	9.9903 	10.0134	10.2902	9.89123	14.7882	0.299931
4  	51    	0         	0.02    	6 	10.2712	9.92646	9.9903 	10.1436	9.89123	14.5664	0.217108
5  	66    	1         	0.02    	6 	10.4507	9.89263	9.93845	10.4552	9.89123	21.0309	0.562528
6  	64    	2         	0.02    	6 	10.2903	9.89263	9.90243	10.23  	9.89123	13.8308	0.337378
7  	74    	3         	0.02    	6 	10.2472	9.89123	9.89263	9.95916	9.89123	13.8308	0.0679303
8  	125   	4         	0.973616	6 	13.6706	10.2383	11.2193	16.3214	9.87397	31.8336	6.08317  
9  	65    	0         	0.02    	6 	10.2571	9.89123	9.96971	10.4738	9.87397	14.0761	0.582582 

88 	113   	15        	0.786404	6 	12.5633	9.90508	10.7907	13.4966	9.78676	29.6634	3.59156    
89 	109   	16        	0.526547	6 	12.6499	9.78676	10.2562	12.5097	9.78676	28.7284	2.72289    
90 	105   	17        	0.641058	6 	12.8427	9.78676	10.3797	13.945 	9.78676	30.7095	4.15824    
91 	88    	18        	0.5     	6 	11.8164	9.78676	9.8354 	11.682 	9.78676	31.126 	1.89524    
92 	111   	19        	0.750162	6 	13.8933	9.78676	10.8849	16.3861	9.78676	38.591 	6.59935    
93 	106   	20        	0.5     	6 	12.0165	9.78676	9.97317	11.8169	9.78676	28.3219	2.03016    
94 	115   	21        	0.732377	6 	13.4281	9.78676	10.6979	14.7338	9.78676	28.8417	4.94702    
95 	88    	22        	0.5     	6 	12.0201	9.78676	9.94783	11.3476	9.78676	25.8701	1.56085    
96 	118   	23        	0.794242	6 	12.9595	9.87493	10.9381	14.2567	9.78676	27.6554	4.38173    
97 	104   	24        	0.5     	6 	12.657 	9.78676	10.238 	13.2294	9.78676	32.7177	3.44262    
98 	87    	25        	0.54618 	6 	12.5643	9.78676	10.0251	11

176	104   	26        	0.564561	6 	12.4177	9.78676	10.3678	12.8786	9.78676	29.4855	3.09185    
177	107   	27        	0.788892	6 	13.1549	9.78676	10.4248	14.732 	9.78676	32.4411	4.94523    
178	110   	28        	0.662345	6 	13.2094	9.81073	10.5603	14.7189	9.78676	31.605 	4.90817    
179	110   	29        	0.664875	6 	13.7166	9.84703	10.9229	15.3964	9.78676	33.8133	5.54935    
180	105   	30        	0.621096	6 	12.9564	9.78676	10.4787	13.2655	9.78666	33.2853	3.4787     
181	61    	0         	0.02    	6 	9.96161	9.78676	9.78676	9.78676	9.78666	19.695 	0          
182	61    	1         	0.02    	6 	9.91379	9.78676	9.78676	9.78676	9.78666	23.7926	0          
183	66    	2         	0.02    	6 	9.92684	9.78666	9.78676	9.78676	9.78666	24.8963	0.000105802
184	72    	3         	0.02    	6 	9.78667	9.78666	9.78666	9.78666	9.78666	9.78676	0          
185	128   	4         	1       	6 	14.3218	10.2963	11.4332	16.8317	9.78666	32.6107	6.53545    
186	97    	5         	0.553766	6 	12.9007	10.0052	10.3313	13

264	63    	0         	0.02    	6 	10.1439	9.86626	9.86626	9.86626	9.84579	23.2868	0          
265	64    	1         	0.02    	6 	10.0296	9.86013	9.86626	9.86626	9.84579	12.9845	0.00612456 
266	61    	2         	0.02    	6 	9.9498 	9.84579	9.86013	9.86626	9.84579	10.7469	0.0204647  
267	70    	3         	0.02    	6 	9.97451	9.84579	9.84579	9.86013	9.84579	14.929 	0.0143401  
268	128   	4         	0.999057	6 	13.9678	10.194 	11.7275	15.4259	9.84579	31.8759	5.23185    
269	99    	5         	0.656123	6 	12.5668	9.92176	10.3402	12.185 	9.84579	27.5085	2.26327    
270	119   	6         	0.851241	6 	13.8242	10.0339	11.9572	15.2365	9.83931	30.9902	5.20256    
271	67    	0         	0.02    	6 	10.5874	9.84579	9.84579	10.5402	9.83931	21.8116	0.694358   
272	58    	1         	0.02    	6 	9.86427	9.84579	9.84579	9.84579	9.83931	11.2655	0          
273	73    	2         	0.02    	6 	10.1646	9.83931	9.84255	9.84579	9.83931	30.7323	0.006487   
274	60    	3         	0.02    	6 	9.88292	9.83931	9.83931	9.

352	63    	3         	0.02    	6 	10.0858	9.78361	9.78361	9.79185	9.78361	25.3114	0.00824249 
353	128   	4         	0.999458	6 	14.6741	10.3816	11.7825	18.8897	9.78361	34.4363	8.5081     
354	95    	5         	0.5     	6 	13.1409	9.88926	10.5226	14.156 	9.78361	29.9969	4.26677    
355	102   	6         	0.719555	6 	12.4983	9.78361	10.6499	12.6334	9.78361	28.6212	2.84979    
356	120   	7         	0.81269 	6 	13.3277	10.0623	11.3519	14.9281	9.78361	35.9359	4.86581    
357	102   	8         	0.680181	6 	13.3801	9.8373 	10.5286	14.7583	9.78361	32.361 	4.92103    
358	115   	9         	0.676552	6 	13.4669	10.0216	10.7925	14.0658	9.78361	34.5269	4.04421    
359	111   	10        	0.734183	6 	13.2927	9.90369	10.93  	14.3277	9.78361	29.6253	4.42399    
360	100   	11        	0.709221	6 	13.65  	9.88682	10.9608	15.6705	9.78361	34.4169	5.78363    
361	105   	12        	0.619855	6 	13.1504	9.79465	10.9993	13.5209	9.78361	41.3174	3.72625    
362	109   	13        	0.755082	6 	12.796 	9.78361	10.9731	14

...

9868	119   	10        	0.752855 	6 	13.0269	10.0783	11.1864	13.6658	9.76411	31.4446	3.58758    
9869	121   	11        	0.84155  	6 	13.7346	10.4065	11.6093	15.8077	9.76411	27.7234	5.40126    
9870	106   	12        	0.761447 	6 	13.2499	10.2721	11.1703	14.1515	9.76411	33.1859	3.87941    
9871	115   	13        	0.828661 	6 	13.7832	10.2966	11.5035	16.1633	9.76411	36.0087	5.86667    
9872	108   	14        	0.740892 	6 	12.5442	9.94243	10.6047	13.0382	9.76411	29.778 	3.09576    
9873	121   	15        	0.863272 	6 	14.048 	10.2001	11.6981	15.6169	9.76411	36.4917	5.41678    
9874	111   	16        	0.760762 	6 	14.5805	10.2722	11.6379	17.1784	9.76411	42.3799	6.90614    
9875	112   	17        	0.694982 	6 	13.1007	10.2203	11.1611	14.9378	9.76411	29.4878	4.71751    
9876	119   	18        	0.791646 	6 	13.9167	10.166 	11.5291	15.6415	9.76411	36.4206	5.47548    
9877	114   	19        	0.758169 	6 	14.057 	10.4346	11.5379	15.2317	9.76411	35.0696	4.79701    
9878	113   	20        	0.788134 	6 	13.4

9954	115   	15        	0.701451 	6 	14.315 	10.0826	11.5149	16.8101	9.77488	40.262 	6.72744    
9955	103   	16        	0.702875 	6 	13.3439	9.98658	10.823 	13.278 	9.77488	32.7648	3.29138    
9956	120   	17        	0.854632 	6 	13.9651	10.17  	11.623 	15.4381	9.77488	32.7961	5.26807    
9957	112   	18        	0.76733  	6 	13.3038	10.1739	11.2468	14.3774	9.77488	28.5636	4.20349    
9958	113   	19        	0.814348 	6 	13.3128	10.1774	11.2013	13.9884	9.77488	35.5204	3.811      
9959	114   	20        	0.831683 	6 	13.4407	10.2081	11.089 	14.5085	9.77488	33.108 	4.30044    
9960	116   	21        	0.810066 	6 	13.8297	10.3149	11.2636	15.885 	9.77488	31.9257	5.57016    
9961	106   	22        	0.753987 	6 	12.691 	10.0334	10.7544	13.3419	9.77488	31.4423	3.30852    
9962	116   	23        	0.853876 	6 	13.7012	10.1186	11.242 	14.2003	9.77488	33.8241	4.08176    
9963	114   	24        	0.819724 	6 	13.2437	10.1661	11.0542	13.393 	9.77488	32.7679	3.2269     
9964	123   	25        	0.85748  	6 	13.4

The team [33, 690, 521, 340, 172, 587] has 900.26 resources,
and solves the three challenges in 9.706704372722395 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 793, 813, 840, 91, 684] has 900.47 resources,
and solves the three challenges in 9.698632613220353 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 904, 242, 840, 786, 684] has 900.32 resources,
and solves the three challenges in 9.707253513167872 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 384, 86, 620, 749, 351] has 899.95 resources,
and solves the three challenges in 9.718231585777641 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 891, 853, 620, 599, 502] has 900.14 resources,
and solves the three challenges in 9.714436587031386 years.
and has the t

The team [33, 349, 531, 620, 172, 339] has 899.94 resources,
and solves the three challenges in 9.725205272658688 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 904, 577, 840, 478, 347] has 899.84 resources,
and solves the three challenges in 9.753552368421516 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 690, 521, 821, 172, 587] has 899.70 resources,
and solves the three challenges in 9.713235003383694 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 122, 111, 620, 854, 684] has 900.38 resources,
and solves the three challenges in 9.711048348236313 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']
The team [33, 495, 365, 840, 786, 575] has 900.41 resources,
and solves the three challenges in 9.711990411092897 years.
and has the

The team [33, 458, 342, 840, 268, 587] has 900.26 resources,
and solves the three challenges in 9.719314649008997 years.
and has the type makeup: ['scientist', 'data miner', 'engineer', 'analyst', 'developer', 'pmo']


## Solutions Analysis

In [1]:
x1 = [33, 892, 474, 588, 877, 848]
x2 = [33, 582, 642, 932, 206, 848]
x3 = [33, 807, 642, 932, 329, 848]
x4 = [33, 807, 642, 932, 88, 848]
x5 = [33, 234, 852, 798, 906, 946]
x6 = [33, 592, 221, 149, 906, 946]
x7 = [33, 221, 592, 149, 906, 946]
x8 = [33, 445, 176, 810, 573, 598]
x9 = [33, 898, 946, 424, 798, 911]
x10 = [33, 137, 276, 773, 848, 573]
x11 = [33, 203, 549, 505, 848, 535]
x12 = [33, 203, 396, 505, 848, 535]
x13 = [33, 259, 549, 505, 848, 535]
x14 = [33, 847, 737, 911, 848, 535]
x15 = [33, 137, 276, 773, 848, 647]
x16 = [33, 152, 198, 773, 848, 573]
x17 = [33, 203, 396, 956, 848, 535]
x18 = [33, 922, 76, 233, 848, 685]
x19 = [33, 444, 600, 848, 293, 929]
x20 = [33, 175, 422, 848, 443, 500]
x21 = [33, 175, 600, 848, 443, 500]
x22 = [33, 446, 443, 848, 774, 892]
x23 = [33, 549, 188, 848, 512, 136]
x24 = [33, 396, 188, 848, 512, 136]
x25 = [33, 175, 288, 848, 443, 374]
x26 = [33, 549, 413, 848, 512, 136]
x27 = [33, 446, 443, 848, 774, 343]
x28 = [33, 176, 288, 848, 443, 374]
x29 = [33, 902, 541, 598, 919, 922]
x30 = [33, 669, 737, 598, 919, 911]
x31 = [33, 443, 391, 598, 597, 448]
x32 = [33, 929, 443, 848, 422, 175]
x33 = [33, 929, 443, 848, 600, 175]
x34 = [33, 115, 854, 848, 757, 279]
x35 = [33, 175, 674, 848, 443, 947]
x36 = [33, 115, 324, 848, 757, 279]
x37 = [33, 140, 451, 726, 151, 598]
x38 = [33, 356, 785, 815, 748, 848]
x39 = [33, 950, 920, 210, 478, 598]
x40 = [33, 526, 555, 438, 323, 598]
x41 = [33, 76, 576, 578, 671, 946]
x42 = [33, 406, 645, 153, 234, 491]
x43 = [33, 805, 739, 924, 660, 153]



In [16]:
combined = []
combined.extend(x1)
combined.extend(x2)
combined.extend(x3)
combined.extend(x4)
combined.extend(x5)
combined.extend(x6)
combined.extend(x7)
combined.extend(x8)
combined.extend(x9)
combined.extend(x10)
combined.extend(x11)
combined.extend(x12)
combined.extend(x13)
combined.extend(x14)
combined.extend(x15)
combined.extend(x16)
combined.extend(x17)
combined.extend(x18)
combined.extend(x19)
combined.extend(x20)
combined.extend(x21)
combined.extend(x22)
combined.extend(x23)
combined.extend(x24)
combined.extend(x25)
combined.extend(x26)
combined.extend(x27)
combined.extend(x28)
combined.extend(x29)
combined.extend(x30)
combined.extend(x31)
combined.extend(x32)
combined.extend(x33)
combined.extend(x34)
combined.extend(x35)
combined.extend(x36)
combined.extend(x37)
combined.extend(x38)
combined.extend(x39)
combined.extend(x40)
combined.extend(x41)
combined.extend(x42)
combined.extend(x43)


In [18]:
best_teams_raw = pd.DataFrame(combined, columns=['idx'])
best_teams_raw['type'] = best_teams_raw['idx'].apply(lambda p: df.at[p, 'type'])

In [20]:
best_teams_unique = best_teams_raw.drop_duplicates(subset='idx', keep='last')


In [38]:
scientists = [33]
validators = best_teams_unique[best_teams_unique['type'] == 'validator']['idx'].to_list()
engineers = best_teams_unique[best_teams_unique['type'] == 'engineer']['idx'].to_list()
# engineers = [848]
analysts = best_teams_unique[best_teams_unique['type'] == 'analyst']['idx'].to_list()
developers = best_teams_unique[best_teams_unique['type'] == 'developer']['idx'].to_list()
auditors = best_teams_unique[best_teams_unique['type'] == 'auditor']['idx'].to_list()

In [39]:
engineers

[848, 598, 946, 153]

In [43]:
from itertools import product

team_stats = pd.DataFrame(columns=['team', 'restime', 'rsrcs'])

def save_stats(team):
    global data
    teamrsrcs = resources(data, team)
    restime   = resoltime(data, team)

    res_data = {'team': team, 'restime': restime,'rsrcs': teamrsrcs}
    
    return pd.concat([team_stats, pd.DataFrame([res_data])], ignore_index=True)
    
#     return team_stats.append(res_data, ignore_index=True)



# Get the Cartesian product of the lists
for combination in product(scientists, validators, engineers, analysts, developers, auditors):
    team_stats = save_stats(list(combination))
    
team_stats


,team,restime,rsrcs
0,"[33, 206, 848, 877, 474, 588]",9.418664,914.57
1,"[33, 206, 848, 877, 474, 582]",9.670486,890.27
2,"[33, 206, 848, 877, 474, 807]",9.648372,892.32
3,"[33, 206, 848, 877, 474, 592]",9.727454,885.06
4,"[33, 206, 848, 877, 474, 898]",9.381699,918.32
...,...,...,...
1586995,"[33, 739, 153, 805, 924, 950]",10.264443,839.15
1586996,"[33, 739, 153, 805, 924, 526]",10.131456,849.81
1586997,"[33, 739, 153, 805, 924, 578]",10.060897,855.66
1586998,"[33, 739, 153, 805, 924, 234]",9.541456,903.30


In [23]:
len(engineers)*len(validators) * len(analysts) * len(auditors) * len(developers)

1587000

In [41]:
len(validators) * len(analysts) * len(auditors) * len(developers) 

396750

In [44]:
valid = team_stats[team_stats['rsrcs'] <= 900]

In [45]:
valid

,team,restime,rsrcs
1,"[33, 206, 848, 877, 474, 582]",9.670486,890.27
2,"[33, 206, 848, 877, 474, 807]",9.648372,892.32
3,"[33, 206, 848, 877, 474, 592]",9.727454,885.06
5,"[33, 206, 848, 877, 474, 259]",9.805563,878.08
6,"[33, 206, 848, 877, 474, 847]",9.714124,886.27
...,...,...,...
1586993,"[33, 739, 153, 805, 924, 451]",10.097839,852.58
1586994,"[33, 739, 153, 805, 924, 815]",9.614491,896.08
1586995,"[33, 739, 153, 805, 924, 950]",10.264443,839.15
1586996,"[33, 739, 153, 805, 924, 526]",10.131456,849.81


In [46]:
valid.sort_values(by='restime', ascending=True)[valid['restime']< 9.57]

<ipython-input-46-f0ccdcedbb3d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid.sort_values(by='restime', ascending=True)[valid['restime']< 9.57]


,team,restime,rsrcs
646898,"[33, 892, 848, 645, 474, 234]",9.566519,900.00
1429418,"[33, 576, 848, 642, 406, 451]",9.566523,900.00
638610,"[33, 892, 848, 396, 175, 233]",9.566535,900.00
849487,"[33, 902, 848, 500, 136, 188]",9.566547,900.00
1440812,"[33, 576, 848, 645, 785, 188]",9.566633,899.99
...,...,...,...
799188,"[33, 288, 848, 374, 76, 413]",9.569954,899.85
1222605,"[33, 151, 848, 922, 535, 259]",9.569965,899.67
740835,"[33, 343, 848, 877, 446, 233]",9.569966,899.90
751055,"[33, 343, 848, 210, 276, 259]",9.569973,899.88


In [47]:
valid.sort_values(by='restime', ascending=True)[valid['restime']< 9.57].to_csv('my_best.csv', index=False)

<ipython-input-47-ec8a86b3599f>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid.sort_values(by='restime', ascending=True)[valid['restime']< 9.57].to_csv('my_best.csv', index=False)


In [19]:
df.sort_values(by='resources', ascending=True).head(6)

,idx,planet,leadership,type,resources,p0,p1,p2
341,341,Chaiter,no,quant,51.35,0.000,20.540,20.540
119,119,Hegniogawa,no,validator,51.68,20.672,5.168,15.504
426,426,Valmeclite,no,engineer,52.33,0.000,41.864,10.466
820,820,Phodeliv,no,quant,52.35,0.000,20.940,20.940
427,427,Uvarvis,no,scientist,52.68,0.000,10.536,42.144
536,536,Thedrora,no,auditor,52.84,21.136,15.852,5.284


In [20]:
resoltime(data, [341,119,426,820,427,536])

30.8420053007778

In [21]:
resources(data, [341,119,426,820,427,536]) #this should be the max possible resources


313.23

In [22]:
data['typelst']

['pmo',
 'validator',
 'auditor',
 'data miner',
 'analyst',
 'developer',
 'quant',
 'engineer',
 'scientist']

In [28]:
import itertools
# Suppose you have the following list of 9 distinct strings
items = data['typelst']

# You want all combinations of 6 items
combinations = list(itertools.combinations(items, 6))

# This will give you a list of tuples. If you want the combinations as lists instead, you can do:
combinations_as_lists = [list(comb) for comb in combinations]

# len(combinations_as_lists) * 6 * 100

# Now you can print or otherwise use your combinations
for combination in combinations_as_lists:
    print(combination)

['pmo', 'validator', 'auditor', 'data miner', 'analyst', 'developer']
['pmo', 'validator', 'auditor', 'data miner', 'analyst', 'quant']
['pmo', 'validator', 'auditor', 'data miner', 'analyst', 'engineer']
['pmo', 'validator', 'auditor', 'data miner', 'analyst', 'scientist']
['pmo', 'validator', 'auditor', 'data miner', 'developer', 'quant']
['pmo', 'validator', 'auditor', 'data miner', 'developer', 'engineer']
['pmo', 'validator', 'auditor', 'data miner', 'developer', 'scientist']
['pmo', 'validator', 'auditor', 'data miner', 'quant', 'engineer']
['pmo', 'validator', 'auditor', 'data miner', 'quant', 'scientist']
['pmo', 'validator', 'auditor', 'data miner', 'engineer', 'scientist']
['pmo', 'validator', 'auditor', 'analyst', 'developer', 'quant']
['pmo', 'validator', 'auditor', 'analyst', 'developer', 'engineer']
['pmo', 'validator', 'auditor', 'analyst', 'developer', 'scientist']
['pmo', 'validator', 'auditor', 'analyst', 'quant', 'engineer']
['pmo', 'validator', 'auditor', 'analyst',